In [25]:
import pandas as pd
import numpy as np
import random
import pickle
import os

from numpy import exp, sqrt, dot
from scipy.spatial.distance import cdist
from scipy.io import loadmat
from random import shuffle

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor

from scipy.stats import kurtosis
from scipy.stats import moment
from scipy.stats import skew


import tensorflow as tf
from tensorflow.python.client import device_lib

import warnings
warnings.filterwarnings('ignore')

# Kernel mean embedding stuff

In [26]:
class Kernel(object):
    """ Kernel mean embedding class
        Kernel class from Zoltan Szabo
    """

    def __init__(self, par=None):
        """ Initialization.

        Parameters
        ----------
        par : dictionary, optional
              Name of the kernel and its parameters (default is
              {"name": "RBF", "sigma": 1}). The name of the kernel comes
              from "RBF", "exponential", "Cauchy", "student", "Matern3p2",
              "Matern5p2", "polynomial", "ratquadr" (rational quadratic),
              "invmquadr" (inverse multiquadr).
        """
        if par is None:
            par = {"name": "RBF", "sigma": 1}

        name = par["name"]
        self.name = name

        # other attributes:
        if name == "RBF" or name == "exponential" or name == "Cauchy":
            self.sigma = par["sigma"]
        elif name == "student":
            self.d = par["d"]
        elif name == "Matern3p2" or name == "Matern5p2":
            self.l = par["l"]
        elif name == "polynomial":
            self.c = par["c"]
            self.exponent = par["exponent"]
        elif name == "ratquadr" or name == "invmquadr":
            self.c = par["c"]
        else:
            raise Exception("kernel=?")

    def gram_matrix(self, y1, y2):
        """  Compute the Gram matrix = [k(y1[i,:], y2[j,:])]; i, j: running.

        Parameters
        ----------
        y1 : (number of samples1, dimension)-ndarray
             One row of y1 corresponds to one sample.
        y2 : (number of samples2, dimension)-ndarray
             One row of y2 corresponds to one sample.

        Returns
        -------
        g : ndarray.
            Gram matrix of y1 and y2.
        """

        if self.name == "RBF":
            sigma = self.sigma
            g = cdist(y1, y2)
            g = exp(-g ** 2 / (2 * sigma ** 2))
        elif self.name == "exponential":
            sigma = self.sigma
            g = cdist(y1, y2)
            g = exp(-g / (2 * sigma ** 2))
        elif self.name == "Cauchy":
            sigma = self.sigma
            g = cdist(y1, y2)
            g = 1 / (1 + g ** 2 / sigma ** 2)
        elif self.name == "student":
            d = self.d
            g = cdist(y1, y2)
            g = 1 / (1 + g ** d)
        elif self.name == "Matern3p2":
            l = self.l
            g = cdist(y1, y2) 
            g = (1 + sqrt(3) * g / l) * exp(-sqrt(3) * g / l)
        elif self.name == "Matern5p2":
            l = self.l
            g = cdist(y1, y2)
            g = (1 + sqrt(5) * g / l + 5 * g ** 2 / (3 * l ** 2)) * \
                exp(-sqrt(5) * g / l)
        elif self.name == "polynomial":
            c = self.c
            exponent = self.exponent
            g = (dot(y1, y2.T) + c) ** exponent
        elif self.name == "ratquadr":
            c = self.c
            g = cdist(y1, y2) ** 2
            g = 1 - g / (g + c)
        elif self.name == "invmquadr":
            c = self.c
            g = cdist(y1, y2)
            g = 1 / sqrt(g ** 2 + c ** 2)
        else:
            raise Exception("kernel=?")

        return g

In [27]:
# Compute the linear kernel product of 
# the mean embedding of X1 and X2
# denoted as K(i, j) above
def mean_embedding(X1, X2, kernel):
    k = Kernel(kernel)
    gram_mat = k.gram_matrix(X1, X2)
    # Number of instances in the bag
    N = float(gram_mat.shape[0])
    mu_X1_X2 = gram_mat.ravel().sum() / N**2
    return (mu_X1_X2)

# Return a symmetrised matrix
def symmetrise(A):
    return(A + A.T - np.diag(A.diagonal()))

In [28]:
# Compute the Gram matrix K given the kernel and 
# the smoothing parameter theta
def compute_gram_train(df, kernel, theta):
    col_feature = [col for col in df.columns if col != 'id']
    nb_bag = df["id"].nunique()
    K_matrix = np.zeros((nb_bag, nb_bag))
    
    print("Computing {0} Gram matrix for theta={1}:".format(kernel, theta))
    for i in range(nb_bag):
        #if (i%50 == 0):
            #print("Bag number: {0}". format(i))

        for j in range(i+1):
            bag_i = df['id'].unique()[i]
            bag_j = df['id'].unique()[j]
    
            X1 = df.loc[df["id"] == bag_i, col_feature].values
            X2 = df.loc[df["id"] == bag_j, col_feature].values

            K_matrix[i,j] = mean_embedding(X1, X2, {'name': kernel, 'c': theta})
            
    return symmetrise(K_matrix)

def compute_gram_test(df_train, df_test, kernel, theta):
    col_feature = [col for col in df_train.columns if col != 'id']
    nb_bag_train = df_train["id"].nunique()
    nb_bag_test = df_test["id"].nunique()
    K_matrix = np.zeros((nb_bag_train, nb_bag_test))
    
    for i in range(len(df_train['id'].unique())):
        #if (i%50 == 0):
            #print("Bag number: {0}". format(i))
        
        for j in range(len(df_test['id'].unique())):
            bag_i = df_train['id'].unique()[i]
            bag_j = df_test['id'].unique()[j]
            
            X1 = df_train.loc[df_train["id"] == bag_i, col_feature].values
            X2 = df_test.loc[df_test["id"] == bag_j, col_feature].values
            
            K_matrix[i,j] = mean_embedding(X1, X2, {'name': kernel, 'c': theta})
        
    return K_matrix

In [29]:
# Class for kernel ridge regression
class RidgeRegression(object):
    def __init__(self, l2_reg):
        self.l2_reg = l2_reg

    def fit(self, G, y):
        # Train size
        n_train = G.shape[0]
        ridge_mat = G + (self.l2_reg * n_train) * np.identity(n_train)
        self.ridge_mat = ridge_mat
        # Shape of y_train is (1, n_train)
        self.y_train = y

    def predict(self, G):
        y_test_hat = self.y_train.dot(np.linalg.solve(self.ridge_mat, G))
        return y_test_hat

# moments stuff

In [30]:
def get_quantile(series, percentile):
    result = series.quantile([percentile])
 
    return result

In [31]:
def moments_df(full_data, n_moments, quantile=False):
    
    full_data.columns = ['meta_feature', 'id']
    list_operation = ['np.mean', 'np.var', 'skew', 'kurtosis']
    list_quantile = [0.25, 0.5, 0.75]

    dic_df = {}

    for moments in range(1, n_moments+1):
        if moments < 5:
            dic_df[moments] = eval("np.stack(full_data.groupby('id').apply(lambda group: " + 
                                   list_operation[moments-1] + "(group)).values)[:,0]")
        else:
            dic_df[moments] = np.stack(full_data.groupby('id').apply(lambda group: moment(group, moment=moments)))[:,0]
            
    if quantile:
        count = 1
        for quant in list_quantile:
            dic_df[n_moments+count] = full_data.groupby('id').apply(lambda group: get_quantile(group, quant)).iloc[:,0].values
            count += 1    
        n_moments += len(list_quantile)

    array_data = np.reshape(dic_df[1], (dic_df[1].shape[0], 1))
    
    if n_moments > 1:
        for key in range(2, n_moments+1):
            array_data = np.hstack([array_data, np.reshape(dic_df[key], (dic_df[key].shape[0], 1))])   

    df = pd.DataFrame(array_data)
#     if dataset != "CORN" and dataset != "WHEAT":
#          df = pd.concat([df, full_data.groupby('id').mean()[constant_columns].reset_index()], axis=1)
    df['id'] = full_data['id'].unique()
    df.columns = ['moment_'+str(index) for index in range(1,n_moments+1)] + ['id']
    df = df.set_index('id')
    
    return df

# Get the data 

In [32]:
dataset = "CORN"

In [33]:
def load_data():
    
    if dataset == "CORN" or dataset == "WHEAT":
        full_data = pickle.load(open(dataset + ".p", "rb"))
        columns = (["id"] +  ['label'] + ["reflectance_" + str(i) for i in range(92)])
        full_data.columns = columns
        
    else:
        mat_dict = loadmat('../MIR/' + dataset + '.mat')
        full_data = pd.DataFrame(mat_dict[dataset])

        # Rename columns to something more interpretable
        columns = (["id"] + ["reflectance_" + str(i) for i in range(7)]
                   + ["solar_" + str(i) for i in range(5)] + ['label'])
        full_data.columns = columns

        if dataset == "MISR2":
            full_data = full_data[full_data['id'].isin(list(full_data['id'].value_counts().index[full_data['id'].value_counts() == 100]))]

    return full_data

# Algorithm (instance-MIR 2.0)

In [35]:
def instance_stacking(l2=0.1, kernel_parameter=1):

    full_data = load_data()
    random_seed_list = list(range(10))
    
    final_loss_train, final_loss_test = [], []
    count = 1
    
    for index in range(10):
        kf = KFold(n_splits=5, shuffle=True, random_state=random_seed_list[index])
        kf2 = KFold(n_splits=50, shuffle=True)
        cols_exclude = ["id", "label"]
        features = [col for col in list(full_data.columns) if col not in cols_exclude]
        list_loss_train, list_loss_test = [], []

        for train_index, test_index in kf.split(list(full_data['id'].unique())):
            train_index, test_index = np.array(full_data['id'].unique())[list(train_index)], np.array(full_data['id'].unique())[list(test_index)]

            fold_number = 0
            dic_val_x, dic_val_y = {}, {}
            for train_index2, val_index in kf2.split(list(train_index)):

                train_index2 = train_index[list(train_index2)]
                val_index = train_index[list(val_index)]
                
                train = full_data[full_data['id'].apply(lambda value: value in train_index2)]
                val = full_data[full_data['id'].apply(lambda value: value in val_index)]
                test = full_data[full_data['id'].apply(lambda value: value in test_index)]
            
                scaler = StandardScaler()
                scaler.fit(train[features])
                train[features], val[features], test[features] = scaler.transform(train[features]), scaler.transform(val[features]), scaler.transform(test[features])

                train_x, train_y = train[features], train['label']
                val_x, val_y = val[features], val['label']
                test_x, test_y = test[features], test['label']
                
                #mlp = MLPRegressor(hidden_layer_sizes=(128,), learning_rate_init=0.001, max_iter=100, alpha=0.05)
                mlp = MLPRegressor(hidden_layer_sizes=(512,), learning_rate_init=0.001, max_iter=75, alpha=1)
                mlp.fit(train_x, train_y)
                train_pred = mlp.predict(train_x)
                val_pred = mlp.predict(val_x)
                test_pred = mlp.predict(test_x)
            
                df_val_pred = pd.DataFrame(np.concatenate([np.reshape(val_pred, (val_pred.shape[0],1)), 
                         np.reshape(val['id'].values, (val_pred.shape[0],1))], axis=1))
    
                df_test_pred = pd.DataFrame(np.concatenate([np.reshape(test_pred, (test_pred.shape[0],1)), 
                         np.reshape(test['id'].values, (test_pred.shape[0],1))], axis=1))
            
                # truth
                true_val_y = val.groupby(['id']).mean()['label'].values
                true_test_y = test.groupby(['id']).mean()['label'].values
                
                dic_val_x[fold_number], dic_val_y[fold_number] = df_val_pred, true_val_y
                fold_number += 1
            
            arr_val_y = np.concatenate([dic_val_y[x] for x in dic_val_y])
            df_val_x = pd.DataFrame(np.concatenate([dic_val_x[x] for x in dic_val_x]))
            df_val_x.columns, df_test_pred.columns = ['feature', 'id'], ['feature', 'id']
            
################################################################################################################            
            # kernel mean embedding + krr stuff
            kernel_embedded_matrix_train = compute_gram_train(df_val_x, 'invmquadr', kernel_parameter)
            kernel_embedded_matrix_test = compute_gram_test(df_val_x, df_test_pred, 'invmquadr', kernel_parameter)
            krr = RidgeRegression(l2_reg=l2)
            krr.fit(kernel_embedded_matrix_train, arr_val_y)
            y_val_pred = krr.predict(kernel_embedded_matrix_train)
            y_test_pred = krr.predict(kernel_embedded_matrix_test)
################################################################################################################
            
#             df_val_x = moments_df(df_val_x, 3, True)
#             df_test_pred = moments_df(df_test_pred, 3, True)
#             scaler = StandardScaler()
#             scaler.fit(df_val_x)
#             df_val_x, df_test_pred = scaler.transform(df_val_x), scaler.transform(df_test_pred)
           
#             regr = Ridge(alpha = 0.01)
#             regr.fit(df_val_x, arr_val_y)
#             y_val_pred = regr.predict(df_val_x)
#             y_test_pred = regr.predict(df_test_pred)
            
            training_loss = np.sqrt(mean_squared_error(np.reshape(y_val_pred,(y_val_pred.shape[0],1)), 
                                                           np.reshape(arr_val_y, (y_val_pred.shape[0],1))))

            testing_loss = np.sqrt(mean_squared_error(np.reshape(y_test_pred,(y_test_pred.shape[0],1)), 
                                                          np.reshape(true_test_y, (y_test_pred.shape[0],1))))
 
            list_loss_train.append(training_loss)
            list_loss_test.append(testing_loss)
        
            print(training_loss, testing_loss)
            
        final_loss_train.append(np.mean(list_loss_train))
        final_loss_test.append(np.mean(list_loss_test))
        print('Iteration number ' + str(count) + ' is done')
        count += 1

        print('The training loss is ' + str(np.mean(final_loss_train)))
        print('The val loss is ' + str(np.mean(final_loss_test)))


In [ ]:
for l2 in [1/10**8]:
    for kernel_param in [100]:
        print("FOR L2=" + str(l2) + " AND PARAM=" + str(kernel_param))
        instance_stacking(l2, kernel_param)

FOR L2=1e-08 AND PARAM=100
Computing invmquadr Gram matrix for theta=100:
24.237874048842333 23.23341972581816
Computing invmquadr Gram matrix for theta=100:
23.9479348057113 26.157321713565533
Computing invmquadr Gram matrix for theta=100:
23.981932960411307 27.078980075564754
Computing invmquadr Gram matrix for theta=100:
23.83734083837694 24.75818797324095
Computing invmquadr Gram matrix for theta=100:
25.048375810811937 22.250640376579774
Iteration number 1 is done
The training loss is 24.21069169283076
The val loss is 24.695709972953836
Computing invmquadr Gram matrix for theta=100:
23.409911545943615 25.696468302269977
Computing invmquadr Gram matrix for theta=100:
23.914025885186756 25.27968670748308
Computing invmquadr Gram matrix for theta=100:
23.491426743687942 27.41497849815688
Computing invmquadr Gram matrix for theta=100:
25.04801744343425 23.737501595685572
Computing invmquadr Gram matrix for theta=100:
24.840972761755307 22.519360884915585
Iteration number 2 is done
The